<a href="https://colab.research.google.com/github/telmavcosta/data_processing/blob/main/spark_streaming/examples/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [5]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

IllegalArgumentException: Cannot start query with name my_query as a query with that name is already active in this SparkSession

In [9]:
query.isActive

True

In [10]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2025-07-05 08:36:40, 2025-07-05 08:36:45}|49   |
|{2025-07-05 08:36:35, 2025-07-05 08:36:40}|50   |
|{2025-07-05 08:36:30, 2025-07-05 08:36:35}|50   |
|{2025-07-05 08:36:25, 2025-07-05 08:36:30}|50   |
|{2025-07-05 08:36:20, 2025-07-05 08:36:25}|50   |
|{2025-07-05 08:36:15, 2025-07-05 08:36:20}|50   |
|{2025-07-05 08:36:10, 2025-07-05 08:36:15}|50   |
|{2025-07-05 08:36:05, 2025-07-05 08:36:10}|50   |
|{2025-07-05 08:36:00, 2025-07-05 08:36:05}|50   |
|{2025-07-05 08:35:55, 2025-07-05 08:36:00}|50   |
+------------------------------------------+-----+
only showing top 10 rows



In [11]:
query.stop()

# Write output as json

In [ ]:
!rm -rf content/output

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch (TC: customizar o codigo)
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds')
.outputMode('append')
.foreachBatch(save_parquet)
.start()
)


In [ ]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2024-11-22 16:36:45, 2024-11-22 16:36:50}|17   |2       |2024-11-22 16:37:20.187321|
|{2024-11-22 16:36:50, 2024-11-22 16:36:55}|50   |2       |2024-11-22 16:37:20.187321|
|{2024-11-22 16:36:55, 2024-11-22 16:37:00}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:00, 2024-11-22 16:37:05}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:05, 2024-11-22 16:37:10}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:10, 2024-11-22 16:37:15}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:15, 2024-11-22 16:37:20}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:20, 2024-11-22 16:37:25}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:25, 2024-11-22 16:37:30}

In [ ]:
query.stop()

# Enrich data with faker

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.8 MB/s eta 0:00:00


In [ ]:
!rm -rf content/output/events

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def insert_into_table(df, batch_id):
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  df.write.mode("append").format("parquet").save("content/output/events")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_into_table)
.start()
)

In [ ]:
query.stop()


In [ ]:
spark.read.parquet("content/output/events").show(100, False)

+-----------------------+-----+------------------+-----------------------------------------------------------+----------------------------+----------+---------------------+
|timestamp              |value|name              |address                                                    |email                       |dob       |phone                |
+-----------------------+-----+------------------+-----------------------------------------------------------+----------------------------+----------+---------------------+
|2024-11-22 17:09:40.928|13   |Jennifer Padilla  |4723 Justin Stravenue Apt. 795\nSouth Ashleyville, CT 67628|michaeltorres@example.com   |1916-02-22|+1-868-292-8492x48606|
|2024-11-22 17:09:43.928|16   |Brandon Camacho   |88045 Owens Points Apt. 983\nEast Michelleborough, GU 99123|torresedward@example.net    |1932-09-15|(772)402-8764        |
|2024-11-22 17:09:29.928|2    |Timothy Lopez     |311 Church Throughway Suite 666\nDominguezbury, MH 74128   |gloverlaura@example.net  

In [ ]:
query.stop()